# Goodness of fit estimation for GW events

### Imports

In [1]:
#####################
## General purpose ##
#####################
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import chi2

# ###########
# ## Bilby ##
# ###########
# import bilby
# from bilby.gw.detector import InterferometerList
# from bilby.gw.likelihood import GravitationalWaveTransient
# from bilby.gw.waveform_generator import WaveformGenerator
# from bilby.gw.source import lal_binary_black_hole
# logger = bilby.core.utils.logger


# ###########
# ## GWOSC ##
# ###########
# from gwosc.datasets import event_gps
# from gwosc.api import fetch_event_json
# from gwosc.locate import get_event_urls
# from gwosc import datasets


#############
## Getdist ##
#############
from getdist import MCSamples
from getdist import plots


###########
## Other ##
###########
from gwpy.timeseries import TimeSeries
from pesummary.gw.fetch import fetch_open_samples
from pesummary.io import read
from pathlib import Path


###########
## Utils ##
###########
import utils as gwut
from tensiometer.utilities import KL_decomposition, from_confidence_to_sigma

/home/stan/Documents/python_envs/genova/lib/python3.12/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


### Model and parameters
We assume a model parametrization with the following 15 parameters:

| Parameter             | Periodic? | Periodic Range |
| --------------------- | --------- | -------------- |
| `chirp_mass`          | No        | –              |
| `azimuth`             | No        | –              |
| `zenith`              | No        | –              |
| `phase`               | Yes       | \[0, 2π]       |
| `psi`                 | Yes       | \[0, π]        |
| `theta_jn`            | No        | –              |
| `mass_ratio`          | No        | –              |
| `a_1`                 | No        | –              |
| `a_2`                 | No        | –              |
| `tilt_1`              | No        | –              |
| `tilt_2`              | No        | –              |
| `phi_12`              | Yes       | \[0, 2π]       |
| `phi_jl`              | Yes       | \[0, 2π]       |
| `luminosity_distance` | No        | –              |
| `geocent_time`        | No        | –              |

#### Parameter names

In [2]:
MODEL_PARAMS = ["chirp_mass",
                "ra",
                "dec",
                # "azimuth",   # Equivalent of ra in some param estimations
                # "zenith",    # Equivalent of dec in some param estimations
                "phase",
                "psi",
                "theta_jn",
                "mass_ratio",
                "a_1",
                "a_2",
                "tilt_1",
                "tilt_2",
                "phi_12",
                "phi_jl",
                "luminosity_distance",
                "geocent_time",
                ]

# Periodicity of the params
PERIOD_PARAM_DICT = {"phi_12": [0, 2*np.pi],
                     "phi_jl": [0, 2*np.pi],
                     "phase":[0, 2*np.pi],
                     "psi": [0, np.pi],
                     "ra": [0, 2*np.pi],
                     # "dec": [-np.pi/2, np.pi/2],  # Should not be periodic???
                    }

#### Event under consideration
We will define the model later, based on which prior samples are available

We can put any event here, and the rest of the notebook will take care of (down)loading the data, and getting the different $Q$ estimators

In [3]:
EVENT = "GW190521"
# EVENT = "GW200129_065458"
# EVENT = "GW190412"
# EVENT = "GW151226"

### Fetching event posterior samples

In [4]:
# Check if file is already downloaded
if os.path.exists(f"{EVENT}"):
    print("INFO: Directory already exists. Now loading data...")
    path = Path(f"{EVENT}")
    files = list(path.glob("*.h5"))
    file = [file for file in files if EVENT in file.name][0]
    print("   ", file)
    data = read(file)
else:
    print("INFO: Couldn't find data. Now downloading...")
    # Create directory to store event data in
    os.mkdir(f"{EVENT}")

    # Download data if it did not exist
    data = fetch_open_samples(
        EVENT,
        read_file=True,
        delete_on_exit=False,
        outdir=f"./{EVENT}",
        path=f"{EVENT}.h5"
    )

# Samples and prior information
samples = data.samples_dict
priors = data.priors

calibration = priors["calibration"]
prior_samples = priors["samples"]

INFO: Directory already exists. Now loading data...
    GW190521/GW190521_posterior_samples.h5


2025-06-17  00:07:21 PESummary WARNING : Unable to install 'pycbc'. You will not be able to use some of the inbuilt functions.
2025-06-17  00:07:21 PESummary WARNING : Unable to install 'pycbc'. You will not be able to use some of the inbuilt functions.
2025-06-17  00:07:23 PESummary WARNING : Could not find f_final in input file and one was not passed from the command line. Using 1024.0Hz as default
2025-06-17  00:07:23 PESummary WARNING : Could not find delta_f in input file and one was not passed from the command line. Using 0.00390625Hz as default
2025-06-17  00:07:23 PESummary WARNING : Could not find f_start in input file and one was not passed from the command line. Using 20.0Hz as default


In [5]:
# Loop over models in the prior_samples dict and check for non-empty samples
# Only takes the first non-empty model for now
for model, prior in prior_samples.items():
    if len(prior) > 0:
        print(f"Model '{model}' has prior samples.")
        MODEL = model
        break
else:
    print("No prior samples found for any model.")

Model 'NRSur7dq4' has prior samples.


### MCSamples
We convert the prior and posterior samples to MCSamples objects. For completeness, we also show all available models

In [6]:
available_models = samples.keys()
print("All available models:")
for m in available_models:
    if m == MODEL:
        print("   ", m, "<--- Currenty using")
        continue
    print("   ", m)

getdist_settings = {
    'smooth_scale_1D': 0.3,
    'smooth_scale_2D': 0.4,
    'boundary_correction_order': 1,
    'mult_bias_correction_order': 1,
    'ignore_rows': 0.3
    }


# Create MCSamples objects for posterior and prior
mcs_posterior = MCSamples(samples = samples[MODEL].samples.T, names = samples[MODEL].keys(), label="Posterior", settings=getdist_settings)
mcs_prior = MCSamples(samples = prior_samples[MODEL].samples.T, names = prior_samples[MODEL].keys(), label="Prior", settings=getdist_settings)

# mcs_posterior.updateSettings(getdist_settings)
# mcs_posterior.updateBaseStatistics()

# mcs_prior.updateSettings(getdist_settings)
# mcs_prior.updateBaseStatistics()

All available models:
    IMRPhenomPv3HM
    NRSur7dq4 <--- Currenty using
    SEOBNRv4PHM
Removed 0.3 as burn in
Removed 0.3 as burn in


### GoF preparation
To determine a GoF, we need the effective number of freedoms, as well as the prior and posterior covariances $\mathcal{C}_p$ and $\mathcal{C}_\Pi$. For this, we need to take into account the periodicity of some of the parameters, which we achieve using a utilities script from Giulia

In [7]:
# Effective number of freedoms
Neff = gwut.get_Neff_with_periodic_boundaries(mcs_posterior, mcs_prior, MODEL_PARAMS, PERIOD_PARAM_DICT)

# Covariance matrices
Cpi = gwut.covariance_with_periodic_params(mcs_prior, MODEL_PARAMS, PERIOD_PARAM_DICT)
Cp = gwut.covariance_with_periodic_params(mcs_posterior, MODEL_PARAMS, PERIOD_PARAM_DICT)

# Inverses
Cpi_inv = np.linalg.inv(Cpi)
Cp_inv = np.linalg.inv(Cp)

# Data covariance matrix
C_inv = Cp_inv - Cpi_inv
C = np.linalg.inv(C_inv)

# Get means
mu_posterior = gwut.circular_mean_ND(mcs_posterior, MODEL_PARAMS, PERIOD_PARAM_DICT)
mu_prior = gwut.circular_mean_ND(mcs_prior, MODEL_PARAMS, PERIOD_PARAM_DICT)

### Quadratic forms
#### Prior compatibility

In [8]:
# Difference between prior and posterior means
delta_theta_pi = mu_posterior - mu_prior

# Decompose using KL
eig, phi = KL_decomposition(Cpi, Cp)

# Need to use phi.T for some reason?
# We don't know why??
delta_p = phi.T @ delta_theta_pi

# Throw away everything below 1.2
MASK = eig > 1.2
number_thrown_away = np.sum(~MASK)

# Calculate the quadratic form
Q_pi = np.sum(delta_p[MASK]**2 / (eig[MASK] - 1))

p_value = 1 - chi2.cdf(Q_pi, df=len(MODEL_PARAMS) - number_thrown_away)

print("Effecve sigma:", from_confidence_to_sigma(1-p_value))

Effecve sigma: 2.6297686705620396


#### Maximum a Posteriori 
Need to calulate $Q_\mathrm{MAP}$:
* $\theta_p$
* 

#### Maximum Likelihood

In [9]:
PSD = []

for det in calibration[MODEL].detectors:
    strain = data.psd[MODEL][det].strains
    PSD.append(strain)

PSD = np.concatenate(PSD)
Sigma = np.diag(PSD)
Sigma_det = np.prod(PSD)

print(np.sum(np.log(PSD)))
print(PSD.shape)


# TODO: normalization constants (d * ln(2pi) - ln|Sigma|), check if correct???
# Factor -2 is now -1??? Is log_likelihood actually chi2 / 2??
# Need to check further...
Q_ML =  -1 * samples[MODEL]["log_likelihood"].max() # - np.sum(np.log(PSD)) - len(PSD) * np.log(2*np.pi)
print(Q_ML)
p_value = 1 - chi2.cdf(Q_ML, df=len(PSD) - len(MODEL_PARAMS))
print(p_value)

print("Effecve sigma:", from_confidence_to_sigma(1-p_value))

-1280465.3197104363
(12288,)
11903.911397937623
0.9912393700455204
Effecve sigma: 0.010980041997812716
